In [1]:
# this is a code cell that contains Python code
# we usually start with the imports
# these are the imports we usually use for machine learning
import numpy as np
import scipy
import scipy.sparse as sps
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import tensorflow as tf
from nltk.corpus import movie_reviews

In [2]:
num_neurons = 20
num_features = 5000

In [3]:
# getting the data, like last time

corpus, targets = zip(*[(movie_reviews.raw(fileid), category)
                         for category in movie_reviews.categories() for fileid in movie_reviews.fileids(category)])

count_vectorizer = CountVectorizer(stop_words='english', max_df=0.95, min_df=2, max_features=num_features)
bows = count_vectorizer.fit_transform(corpus)

# convert targets to numbers
targets = np.array([0 if target == 'neg' else 1 for target in targets])

X_train, X_test, y_train, y_test = train_test_split(bows, targets, test_size=0.1, shuffle=True)

# the problem: we have sparse arrays, but neural network need dense arrays!
# the solution will be word embeddings, here we just convert to dense arrays
X_train = X_train.toarray()
X_test = X_test.toarray()

In [4]:
# we are building a computational graph, tensors are not evaluated beforehand
v = tf.zeros((10, 10))
v

<tf.Tensor 'zeros:0' shape=(10, 10) dtype=float32>

In [5]:
v + 2

<tf.Tensor 'add:0' shape=(10, 10) dtype=float32>

In [6]:
# variables are for the parameters (weights) for the NN
W1 = tf.Variable(tf.random_normal((num_neurons, num_features), stddev=0.01), name='W1')
b1 = tf.Variable(tf.random_normal((num_neurons, 1), stddev=0.01), name='b1')
W2 = tf.Variable(tf.random_normal((1, num_neurons), stddev=0.01), name='W2')
b2 = tf.Variable(tf.random_normal((1, 1), stddev=0.01), name='b2')
W1, b1, W2, b2

Instructions for updating:
Colocations handled automatically by placer.


(<tf.Variable 'W1:0' shape=(20, 5000) dtype=float32_ref>,
 <tf.Variable 'b1:0' shape=(20, 1) dtype=float32_ref>,
 <tf.Variable 'W2:0' shape=(1, 20) dtype=float32_ref>,
 <tf.Variable 'b2:0' shape=(1, 1) dtype=float32_ref>)

In [7]:
# placeholders these are for the input data
X = tf.placeholder(tf.float32, shape=(num_features, None))
y = tf.placeholder(tf.float32, shape=(1, None))
X

<tf.Tensor 'Placeholder:0' shape=(5000, ?) dtype=float32>

In [ ]:
# the network itself
z = tf.add(tf.matmul(W1, X), b1)
print(z)
a = tf.nn.relu(z)
print(a)
z = tf.add(tf.matmul(W2, a), b2)
print(z)
#a = tf.nn.sigmoid(z) # this is already in the loss
a = z
print(a)

In [ ]:
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a, labels=y))

In [ ]:
loss

In [ ]:
num_epochs = 10

In [ ]:
# we need an optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [ ]:
def minibatches(X, y, minibatch_size):

# shuffling
    perm = np.random.permutation(X.shape[0])
    X_shuffled = X[perm]
    y_shuffled = y[perm]
    
    mini_batches = [(X[i*minibatch_size:(i+1)*minibatch_size], y[i*minibatch_size:(i+1)*minibatch_size])
                   for i in range(X.shape[0] // minibatch_size)]
# we lost some examples at the end, doesn't really matter for this example
    return mini_batches

In [ ]:
minibatch_size = 32
# we need a Session to run the graph
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print("Loss at start: {}".format(sess.run(loss, feed_dict={X: X_train.T, y: y_train[None, :]})))
    for epoch in range(num_epochs):
        # minibatch training
        epoch_loss = 0
        num_minibatches = X_train.shape[0] // minibatch_size
        for X_mini, y_mini in minibatches(X_train, y_train, minibatch_size):
            # important: we have to transpose here! each example is in a column
            _, mini_loss = sess.run([optimizer, loss], feed_dict={X: X_mini.T, y: y_mini[None, :]})
            epoch_loss += mini_loss
        epoch_loss /= num_minibatches
        predictions = sess.run(a, feed_dict={X: X_train.T, y: y_train[None, :]})
        predictions = np.array([0 if pred < 0 else 1 for pred in predictions.ravel()])
        accuracy = (y_train == predictions).sum() / len(y_train)
        print("Loss in epoch {}: {}, accuracy: {}".format(epoch, epoch_loss, accuracy))
    # accuracy on test set
    predictions = sess.run(a, feed_dict={X: X_test.T, y: y_test[None, :]})
    predictions = np.array([0 if pred < 0 else 1 for pred in predictions.ravel()])
    accuracy = (y_test == predictions).sum() / len(y_test)
    print("Accuracy on test set: {}".format(accuracy))
    tf.summary.FileWriter('nn_graph', sess.graph)